In [1]:
import os
import sys
import re
from datetime import datetime, timedelta
from dateutil.rrule import rrule, WEEKLY, FR, SU
sys.path.append('/Users/laurenthericourt/projets/trading/trading')

import psycopg2
import pandas as pd
from pandas.io.sql import read_sql
pd.set_option('display.max_rows', 500)
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

from config.load import load_conf
from db.utils import get_uri_db
from data.candle import SYMBOLS
from indicator.trend import BollingerBands, Adx
from indicator.oscillator import Macd, Rsi, Stochastic

In [2]:
load_conf('../config/configuration.yaml')

In [3]:
dsn = get_uri_db()
schema = 'trading'

# Get data
## Candles

In [4]:
start_date = '2020-11-01'
end_date = '2020-12-01'

In [5]:
candles = pd.DataFrame()
with psycopg2.connect(dsn) as conn:
    for table in ['candle', 'candle15m', 'candle30m', 'candle1h', 'candle4h', 'candle1d']:
        sql = f'set search_path = {schema};'
        sql += f'''
            SELECT '{table}' as table, symbol, date, open, close, low, high
            FROM {table}
            WHERE date >= %(start_date)s
              AND date < %(end_date)s
            ORDER BY date ASC;
        '''
        candles_tmp = read_sql(sql, conn, params={'start_date': start_date, 'end_date': end_date})
        candles = pd.concat([candles, candles_tmp])

## Event

In [6]:
COUNTRY_1, COUNTRY_2 = SYMBOL.split('/')

NameError: name 'SYMBOL' is not defined

In [ ]:
with psycopg2.connect(dsn) as conn:
    sql = f'set search_path = {schema};'
    sql += '''
        SELECT date, country, name, actual_value, forecast_value, is_positive
        FROM event
        WHERE importance = 3
        AND country = ANY(%(countries)s::text[])
    '''
    event = read_sql(sql, conn, params={'countries': [COUNTRY_1, COUNTRY_2]})

In [ ]:
months = ['(jan.)', '(févr.)', '(mar)', '(avr)', '(mai)', '(juin)', '(juill.)', '(août)', '(sept.)', '(oct.)', '(nov.)', '(déc)']
regex_month = re.compile(r'|'.join(months).replace('(', '\(').replace(')', '\)').replace('.', '\.'))

periods = [' (t1)', ' (t2)', ' (t3)', ' (t4)', ' m1', ' m2', ' m3', ' m4', ' m5', ' m6', ' m7', ' m8', ' m9', ' m10', ' m11', ' m12']
regex_period = re.compile(r'|'.join(periods).replace('(', '\(').replace(')', '\)'))

In [12]:
event['processed_name'] = event['name'].str.lower()
event['processed_name'] = event['processed_name'].str.replace(regex_month, '')
event['processed_name'] = event['processed_name'].str.replace(regex_period, '')
event['processed_name'] = event['processed_name'].str.strip()

In [13]:
event['processed_name'].value_counts()[:10]

stocks de pétrole brut                              571
inscriptions hebdomadaires au chômage               571
ipc (annuel)                                        263
discours de draghi, président de la bce             209
promesses de ventes de logements (mensuel)          132
rapport jolts - nouvelles offres d'emploi           132
indice pmi non manufacturier de l'ism               132
créations d'emplois dans le secteur non agricole    132
indice pmi manufacturier de l'ism                   132
taux de chômage                                     132
Name: processed_name, dtype: int64

# Show data
## Dates to hide

In [7]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

def compute_datetime_to_hide(start, end):
    start_dt = datetime.strptime(start, '%Y-%m-%d')
    end_dt = datetime.strptime(end, '%Y-%m-%d')
    
    fridays = list()
    for date in rrule(WEEKLY, byweekday=FR, dtstart=start_dt, until=end_dt):
        fridays.append(date)
    sundays = list()
    for date in rrule(WEEKLY, byweekday=SU, dtstart=start_dt, until=end_dt):
        sundays.append(date)
    
    date_ranges = list()
    if fridays[0] > sundays[0]:
        del sundays[0]
    if fridays[-1] > sundays[-1]:
        del fridays[-1]
    
    datetime_to_hide = list()
    for fri, sun in zip(fridays, sundays):
        res = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in datetime_range(fri.replace(hour=22),
                                                                         sun.replace(hour=19, minute=30),
                                                                         timedelta(minutes=5))]
        datetime_to_hide.extend(res)
    return datetime_to_hide

## Methods to plot result

In [8]:
def filter_data(candles, table, symbol=None, start_date=None, end_date=None):
    mask = (candles['table'] == table)
    if symbol:
        mask = mask & (candles['symbol'] == symbol)
    if start_date and end_date:
        mask = mask & (candles['date'] >= start_date) & (candles['date'] < end_date)
    candles_to_show = candles[mask]
    return candles_to_show


def show_candle(candles):
    layout = go.Layout(
        autosize=True,
        width=1400,
        height=800,
        xaxis=go.layout.XAxis(linecolor = 'black',
                              linewidth = 1,
                              mirror = True),
        xaxis2=go.layout.XAxis(linecolor = 'black',
                               linewidth = 1,
                               mirror = True),
        yaxis=go.layout.YAxis(linecolor = 'black',
                              linewidth = 1,
                              mirror = True,
                              domain=[0, 0.2]),
        yaxis2=go.layout.YAxis(linecolor = 'black',
                               linewidth = 1,
                               mirror = True,
                               domain=[0.3, 1]),

    )
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    fig.update_layout(layout, xaxis2_rangeslider_visible=False)
    fig.add_trace(go.Candlestick(x=candles['date'],
                                 open=candles['open'],
                                 high=candles['high'],
                                 low=candles['low'],
                                 close=candles['close']),
                   row=2, col=1)
    
    fig.update_yaxes(fixedrange=False)
    #fig.update_xaxes(rangebreaks=[dict(values=compute_datetime_to_hide(start_date, end_date))])

    return fig
    

## Show result

In [9]:
app = JupyterDash()

app.layout = html.Div([
    html.H1(f'Candles between {start_date} and {end_date}'),
    html.Div([
        html.Div([
            html.Label('Symbol:', className='label'),
            dcc.RadioItems(
                id='symbols_radio',
                options=[{'label': x, 'value': x} for x in SYMBOLS],
                value='EUR/USD'
            )], className='col'),
        html.Div([
            html.Label('Period:', className='label'),
            dcc.RadioItems(
                id='period_radio',
                labelStyle={'display': 'inline-block'},
                options=[{'label': '5min', 'value': 'candle'},
                         {'label': '15min', 'value': 'candle15m'},
                         {'label': '30min', 'value': 'candle30m'},
                         {'label': '1h', 'value': 'candle1h'},
                         {'label': '4h', 'value': 'candle4h'},
                         {'label': '1d', 'value': 'candle1d'}],
                value='candle1d'
            )], className='col'),
        html.Div([
            html.Label('Indicators:', className='label'),
            dcc.Checklist(
                id='indicators_checklist',
                options=[{'label': 'bollinger bands', 'value': 'bollinger_bands'},
                        {'label': 'macd', 'value': 'macd'},
                        {'label': 'rsi', 'value': 'rsi'},
                        {'label': 'stochastic', 'value': 'stochastic'},
                        {'label': 'adx', 'value': 'adx'}],
                value=[]
            )], className='col'),
    ], className='row'),
    dcc.Graph(id='candles_graph'),
])

@app.callback(
    Output('candles_graph', 'figure'),
    Input('symbols_radio', 'value'),
    Input('period_radio', 'value'),
    Input('indicators_checklist', 'value'))
def update_figure(selected_symbol, selected_period, selected_indicators):
    filtered_data = filter_data(candles, selected_period, selected_symbol)
    fig = show_candle(filtered_data)
    if 'bollinger_bands' in selected_indicators:
        bb = BollingerBands(filtered_data, 'close')
        bb.compute()
        fig = bb.plot(fig)
    if 'macd' in selected_indicators:
        m = Macd(filtered_data, 'close')
        m.compute()
        fig = m.plot(fig)
    if 'rsi' in selected_indicators:
        r = Rsi(filtered_data, 'close')
        r.compute()
        fig = r.plot(fig)
    if 'stochastic' in selected_indicators:
        s = Stochastic(filtered_data)
        s.compute(slow=True)
        fig = s.plot(fig)
    if 'adx' in selected_indicators:
        a = Adx(filtered_data)
        a.compute()
        fig = a.plot(fig)
    return fig

app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/


# Test indicators

In [6]:
pd.set_option('display.max_rows', 500)

In [6]:
SYMBOL = 'EUR/USD'
candles_for_indicator = candles[(candles['symbol'] == SYMBOL) & (candles['table'] == 'candle1h')]
#candles.set_index('date', inplace=True)

In [19]:
true_range = pd.DataFrame()
true_range['HL'] = candles_for_indicator['high'] - candles_for_indicator['low']
true_range['HC'] = (candles_for_indicator['high'] - candles_for_indicator.shift(1)['close']).abs()
true_range['LC'] = (candles_for_indicator['low'] - candles_for_indicator.shift(1)['close']).abs()
true_range_max = true_range.max(axis=1)

In [32]:
from stockstats import StockDataFrame as Sdf

In [33]:
candles_sdf = Sdf.retype(candles_for_indicator)

In [34]:
adx_sdf = candles_sdf['adx'].tolist()

In [40]:
pd.DataFrame(adx_sdf)

,0
0,NaN
1,100.000000
2,50.315457
3,50.308211
4,58.531385
...,...
516,37.461157
517,43.132447
518,47.933626
519,46.656858


In [44]:
candles_for_indicator['dx_package'] = 0
candles_for_indicator['dx_package'] = adx_sdf

<ipython-input-44-b1e7f547aef3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candles_for_indicator['dx_package'] = 0
<ipython-input-44-b1e7f547aef3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candles_for_indicator['dx_package'] = adx_sdf


In [7]:
from ta.trend import ADXIndicator
adx_ta = ADXIndicator(candles_for_indicator['high'], candles_for_indicator['low'], candles_for_indicator['close']).adx()
adx_neg_ta = ADXIndicator(candles_for_indicator['high'], candles_for_indicator['low'], candles_for_indicator['close']).adx_neg()
adx_pos_ta = ADXIndicator(candles_for_indicator['high'], candles_for_indicator['low'], candles_for_indicator['close']).adx_pos()
candles_for_indicator['adx_ta'] = adx_ta
candles_for_indicator['adx_neg_ta'] = adx_neg_ta
candles_for_indicator['adx_pos_ta'] = adx_pos_ta

/Users/laurenthericourt/.pyenv/versions/3.8.6/lib/python3.8/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/Users/laurenthericourt/.pyenv/versions/3.8.6/lib/python3.8/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
<ipython-input-7-e0dfb30eb75f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candles_for_indicator['adx_ta'] = adx_ta
<ipython-input-7-e0dfb30eb75f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [8]:
a = Adx(candles_for_indicator)
a.compute(14)
candles_for_indicator['adx_pos_mine'] = a.result[0]
candles_for_indicator['adx_neg_mine'] = a.result[1]
candles_for_indicator['adx_mine'] = a.result[2]

<ipython-input-8-32503d1af1c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candles_for_indicator['adx_pos_mine'] = a.result[0]
<ipython-input-8-32503d1af1c1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candles_for_indicator['adx_neg_mine'] = a.result[1]
<ipython-input-8-32503d1af1c1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [9]:
candles_for_indicator[candles_for_indicator['date'] >= '2020-11-23']

,table,symbol,date,open,close,low,high,adx_ta,adx_neg_ta,adx_pos_ta,adx_pos_mine,adx_neg_mine,adx_mine
2992,candle1h,EUR/USD,2020-11-23 00:00:00,1.18663,1.18726,1.18643,1.18741,14.134052,21.213921,25.265345,25.265345,21.213921,14.134052
3000,candle1h,EUR/USD,2020-11-23 01:00:00,1.18726,1.18764,1.18722,1.18786,14.227535,20.154156,27.515726,27.515726,20.154156,14.227535
3008,candle1h,EUR/USD,2020-11-23 02:00:00,1.18764,1.18751,1.18726,1.18771,14.314340,19.419565,26.512816,26.512816,19.419565,14.314340
3016,candle1h,EUR/USD,2020-11-23 03:00:00,1.18751,1.18730,1.18728,1.18765,14.394946,18.812411,25.683891,25.683891,18.812411,14.394946
3024,candle1h,EUR/USD,2020-11-23 04:00:00,1.18730,1.18736,1.18720,1.18738,14.337054,19.225498,25.269968,25.269968,19.225498,14.337054
3032,candle1h,EUR/USD,2020-11-23 05:00:00,1.18736,1.18773,1.18726,1.18777,14.764302,18.324402,27.669733,27.669733,18.324402,14.764302
3040,candle1h,EUR/USD,2020-11-23 06:00:00,1.18773,1.18756,1.18729,1.18794,15.361886,17.216824,27.578109,27.578109,17.216824,15.361886
3048,candle1h,EUR/USD,2020-11-23 07:00:00,1.18756,1.18687,1.18678,1.18795,15.016658,19.982761,24.685774,24.685774,19.982761,15.016658
3056,candle1h,EUR/USD,2020-11-23 08:00:00,1.18687,1.18736,1.18635,1.18764,14.024807,21.461194,21.952088,21.952088,21.461194,14.024807
3064,candle1h,EUR/USD,2020-11-23 09:00:00,1.18736,1.18783,1.18707,1.18834,14.019942,19.206216,25.436920,25.436920,19.206216,14.019942


In [18]:
candles_for_indicator.tail()

,table,symbol,date,open,close,low,high,adx_ta,adx_mine
4124,candle1h,EUR/USD,2020-11-30 19:00:00,1.19526,1.19478,1.19429,1.19531,31.448474,31.448474
4132,candle1h,EUR/USD,2020-11-30 20:00:00,1.19478,1.19282,1.19278,1.19479,31.965668,31.965668
4140,candle1h,EUR/USD,2020-11-30 21:00:00,1.19282,1.19303,1.19244,1.19323,32.618129,32.618129
4148,candle1h,EUR/USD,2020-11-30 22:00:00,1.19303,1.19321,1.19275,1.19378,32.577825,32.577825
4156,candle1h,EUR/USD,2020-11-30 23:00:00,1.19321,1.19380,1.19305,1.19394,32.354675,32.354675


In [7]:
aapl.head(30)

,date,open,high,low,close,tr_max,dm_plus,dm_minus,tr_max_n,dm_plus_n,dm_minus_n,dx
0,29/03/2018,167.81,171.75,166.90,165.2635,4.8500,0.00,0.00,NaN,NaN,NaN,NaN
1,02/04/2018,166.64,168.94,164.47,164.1800,4.4700,0.00,2.43,NaN,NaN,NaN,NaN
2,03/04/2018,167.64,168.75,164.88,165.8643,4.5700,0.00,0.00,NaN,NaN,NaN,NaN
3,04/04/2018,168.88,172.01,164.77,169.0361,7.2400,3.26,0.00,NaN,NaN,NaN,NaN
4,05/04/2018,172.58,174.23,172.08,170.2082,5.1939,2.22,0.00,NaN,NaN,NaN,NaN
5,06/04/2018,170.97,172.48,168.20,165.8545,4.2800,0.00,3.88,NaN,NaN,NaN,NaN
6,09/04/2018,169.88,173.09,169.85,167.4995,7.2355,0.61,0.00,NaN,NaN,NaN,NaN
7,10/04/2018,173.00,174.00,171.53,170.6515,6.5005,0.91,0.00,NaN,NaN,NaN,NaN
8,11/04/2018,172.23,173.92,171.70,169.8536,3.2685,0.00,0.00,NaN,NaN,NaN,NaN
9,12/04/2018,173.41,175.00,173.04,171.5281,5.1464,1.08,0.00,NaN,NaN,NaN,NaN


In [11]:
import numpy as np
aapl[['tr_max', 'dm_plus', 'dm_minus', 'tr_max_n', 'dm_plus_n', 'dm_minus_n']] = np.array(a.result)

ValueError: Must have equal len keys and value when setting with an ndarray

In [32]:
import numpy as np
pd.Series(np.where(delta < 0, delta, 0))

0       0.00000
1       0.00000
2      -0.00012
3       0.00000
4       0.00000
         ...   
9982    0.00000
9983   -0.00055
9984    0.00000
9985    0.00000
9986   -0.00348
Length: 9987, dtype: float64

In [26]:
ma, bb_up, bb_down = bollinger_bands(candles_for_indicator, 'close')

In [20]:
macd_, signal, hist = macd(candles_for_indicator, 'close')

In [21]:
hist

1           NaN
13          NaN
19          NaN
26          NaN
32          NaN
         ...   
90    -0.000371
98    -0.000097
106    0.000361
114    0.001055
122    0.001435
Name: close, Length: 6546, dtype: float64

In [29]:
bb_down

1           NaN
15          NaN
22          NaN
45          NaN
51          NaN
         ...   
170    1.178107
181    1.179303
184    1.180845
194    1.181929
206    1.182523
Name: close, Length: 9987, dtype: float64

In [11]:
type(signal)

pandas.core.series.Series

In [17]:
candles_for_indicator

NameError: name 'candles_for_indicator' is not defined

In [24]:
candles_for_indicator['max'] = candles_for_indicator[['open', 'close', 'low', 'high']].max(axis=1)